Get fields from Lumerical and save in file for later

In [1]:
import lumerical as lum
import numpy as  np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import imp

from scipy.constants import pi

lumapi = imp.load_source("lumapi", "C:/Program Files/Lumerical/2020a/api/python/lumapi.py")
mode = lumapi.MODE("Template_Luis.lms")

In [2]:
um = 1e-6
nm = 1e-9

Main waveguide parameters

In [3]:
h_LN = 700*nm
h_etch = 350*nm
w_ridge = 1.5*um
h_slab = h_LN - h_etch

theta = 60
wg_length = 10*um #Useless in MODE
w_ridge_base = w_ridge + 2*h_etch/np.tan(theta*pi/180)

print('slab = %.3f um' %(h_slab))
print('width at the base = %.3f um' %(w_ridge_base))

slab = 0.000 um
width at the base = 0.000 um


Wavelength range

In [4]:
wl_start = 500*nm
wl_stop = 3*um
Nwl = 6
wl = np.linspace(wl_start, wl_stop, num=Nwl)
print('Wavelength list =', wl/um, ' um')

Wavelength list = [0.5 1.  1.5 2.  2.5 3. ]  um


Setup simulation volume and mesh sizes

In [5]:
w_slab = 20*um
h_margin = 5*um
h_substrate = 5*um
meshsize = 50*nm
finemesh = 10*nm

Materials

In [6]:
material_substrate = "SiO2_analytic"
material_thinfilm = "LN_analytic_MgO_doped_xne"

Draw waveguide and setup solver

In [10]:
lum.draw_wg(mode, material_thinfilm, material_substrate,
              h_LN, h_substrate, h_etch, w_ridge, w_slab, theta, wg_length)
lum.add_fine_mesh(mode, finemesh, h_LN, w_ridge_base, x_factor=1.2, y_factor=1.5)
lum.add_2D_mode_solver(mode, meshsize, h_LN, h_substrate, 
                         w_slab, wg_length, h_margin)

In [ ]:
lum.